# StanfordCoreNLP Exploitation Ambiguous

In [1]:
import random

import stanza
from stanza.server import CoreNLPClient
stanza.download('en') 
import sys, os


# Add neural coref to SpaCy's pipe
nlp = stanza.Pipeline()
doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
doc.sentences[0].print_dependencies()
pass


2020-07-20 15:38:08 INFO: Downloading default packages for language: en (English)...
2020-07-20 15:38:09 INFO: File exists: /Users/sakshiudeshi/stanza_resources/en/default.zip.
2020-07-20 15:38:12 INFO: Finished downloading models and saved to /Users/sakshiudeshi/stanza_resources.
2020-07-20 15:38:12 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-07-20 15:38:12 INFO: Use device: cpu
2020-07-20 15:38:12 INFO: Loading: tokenize
2020-07-20 15:38:12 INFO: Loading: pos
2020-07-20 15:38:13 INFO: Loading: lemma
2020-07-20 15:38:13 INFO: Loading: depparse
2020-07-20 15:38:14 INFO: Loading: ner
2020-07-20 15:38:15 INFO: Done loading processors!


('Barack', '4', 'nsubj:pass')
('Obama', '1', 'flat')
('was', '4', 'aux:pass')
('born', '0', 'root')
('in', '6', 'case')
('Hawaii', '4', 'obl')
('.', '4', 'punct')


In [2]:
os.environ["CORENLP_HOME"] = "/Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0"
text = "Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."
with CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'],
        timeout=30000,
        memory='16G') as client:
    ann = client.annotate(text)

Starting server with command: java -Xmx16G -cp /Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-fdb253e9587c4058.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref


In [3]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']

In [4]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [5]:
filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [6]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [7]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [8]:
action3 = ['the work was unsatisfactory', 'the performance was not up to the mark',
           'the application was approved', 'the application was not approved',
           'the application was under review', 'the work was appreciated', 
           'this behaviour was unacceptable', 'the project scope had been finalised',
           'the meat was eaten', 'a musical instrument was played', 
           'the game was excellent', 'the performance was excellent', 
           'the performance was unsatisfactory', 'the cookies were baked', 
           'the waiting times were estimated', 'the document was signed', 
           'the delivery was made', 'the tournament was excellent', 
           'the concert was unsatisfactory', 'the concert was appreciated']

verb_action[verb_list_p2[0]] = [action3]

In [9]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [10]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [11]:
def predict_clusters(sentence):
    doc = client.annotate(sentence)
#     print((doc.))
    return(doc.corefChain)
#     if doc._.has_coref: 
#         return (doc._.coref_resolved, doc._.coref_clusters)
#     else:
#         return ('', '')

In [12]:
def get_pred_list(pred_str):
    pred_list = []
    for line in pred_str.split('\n'):
        if 'gender' not in line:
            pred_list.append(line)
    return pred_list

In [13]:
def get_pred_equivalence(pred1, pred2):
    pred1_list = get_pred_list(str(pred1))
    pred2_list = get_pred_list(str(pred2))
#     print(pred1_list)
#     print(pred2_list)
    return pred1_list == pred2_list

In [14]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [25]:
import pickle
import numpy as np


with open('saved_pickles/Exploration/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'rb') as handle:
    occupation_pair_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'rb') as handle:
    occupation1_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_error.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_error.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'rb') as handle:
    occupation_pair_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'rb') as handle:
    occupation1_count = pickle.load(handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/Exploration/verb_count.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/Exploration/action_count.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [26]:
# with open('saved_pickles/unique_input1_set.pickle', 'rb') as handle:
#     unique_input1_set = pickle.load(handle)
    
# with open('saved_pickles/unique_input1_error_set.pickle', 'rb') as handle:
#     unique_input1_error_set = pickle.load(handle)

# with open('saved_pickles/occupation_pair_error.pickle', 'rb') as handle:
#     occupation_pair_error = pickle.load(handle)
    
# with open('saved_pickles/occupation1_error.pickle', 'rb') as handle:
#     occupation1_error = pickle.load(handle)
    
# with open('saved_pickles/occupation2_error.pickle', 'rb') as handle:
#     occupation2_error = pickle.load(handle)  
    
# with open('saved_pickles/verb_error.pickle', 'rb') as handle:
#     verb_error = pickle.load(handle)    
    
# with open('saved_pickles/action_error.pickle', 'rb') as handle:
#     action_error = pickle.load(handle)
    
# with open('saved_pickles/occupation_pair_count.pickle', 'rb') as handle:
#     occupation_pair_count = pickle.load(handle)
    
# with open('saved_pickles/occupation1_count.pickle', 'rb') as handle:
#     occupation1_count = pickle.load(handle)
    
# with open('saved_pickles/occupation2_count.pickle', 'rb') as handle:
#     occupation2_count = pickle.load(handle)  
    
# with open('saved_pickles/verb_count.pickle', 'rb') as handle:
#     verb_count = pickle.load(handle)    
    
# with open('saved_pickles/action_count.pickle', 'rb') as handle:
#     action_count = pickle.load(handle)

In [27]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [28]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [29]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [30]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [31]:
# probability_dict = get_probability_dict(occupation_pair_error, occupation_pair_count)
# print(np.shape(list(probability_dict.keys())))

In [32]:
unique_input1_set_exploitation = set()
unique_input1_error_set_exploitation = set()

occupation_pair_error_exploitation = {}

occupation1_error_exploitation = {}

occupation2_error_exploitation = {}

verb_error_exploitation = {}

action_error_exploitation = {}

occupation_pair_count_exploitation = {}

occupation1_count_exploitation = {}

occupation2_count_exploitation = {}

verb_count_exploitation = {}

action_count_exploitation = {}


In [33]:
oc1_probability = get_probability_dict(occupation1_error, occupation1_count)

print(oc1_probability)
print()

error_rate_dict = get_error_rate_dict(occupation1_error, occupation1_count)
print(error_rate_dict)
print()

output_dict = {}
for i in range(100000):
    oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
    update_dict(output_dict, oc1)
print(get_sorted_dict(output_dict))

{'CEO': 0.10617644179751151, 'guard': 0.07561407026138237, 'construction worker': 0.058536987973540754, 'farmer': 0.05585804111956041, 'lawyer': 0.0532550308000087, 'physician': 0.05093161958255711, 'analyst': 0.050462662147784686, 'sheriff': 0.04971098396286237, 'technician': 0.04937920876400011, 'laborer': 0.04930702866348428, 'cook': 0.04893349056754878, 'driver': 0.048748835886161815, 'engineer': 0.04733874559193399, 'janitor': 0.046873880659770974, 'mechanic': 0.044686432895648324, 'carpenter': 0.044235054785591275, 'chief': 0.04230270882683463, 'accountant': 0.040318586071261646, 'manager': 0.03733018964255634}

{'CEO': 0.8712121212121212, 'guard': 0.6204379562043796, 'construction worker': 0.48031496062992124, 'farmer': 0.4583333333333333, 'lawyer': 0.4369747899159664, 'physician': 0.417910447761194, 'analyst': 0.4140625, 'sheriff': 0.40789473684210525, 'technician': 0.4051724137931034, 'laborer': 0.40458015267175573, 'cook': 0.4015151515151515, 'driver': 0.4, 'engineer': 0.3884

In [36]:
def generate_test_sentences(ITERS=3000):
    err_count = 0

    oc1_probability = get_probability_dict(occupation1_error, occupation1_count)
    oc2_probability = get_probability_dict(occupation2_error, occupation2_count)

    for i in range(ITERS):
#         oc1 = random.choice(occupations_1)
#         oc2 = random.choice(occupations_2)
        oc1 = get_weighted_random_choice(occupation1_error, occupation1_count, probablilities_dict=oc1_probability)
        oc2 = get_weighted_random_choice(occupation2_error, occupation2_count, probablilities_dict=oc2_probability)
        verb = random.choice(list(verb_action.keys()))
        action = random.choice(random.choice(verb_action[verb]))
        pronoun = choose_pronoun_type(verb)
        input1 = ("The " + oc1 + " " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 

        input2 = ("The " + oc1 + " " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[1] + " " + action) 

#         input3 = ("The " + oc1 + " " + verb + " "
#                + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
        pred1 = predict_clusters(input1)
        pred2 = predict_clusters(input2)
#         pred3, _ = predict_clusters(input3)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")

        if input1 not in unique_input1_set:
            unique_input1_set_exploitation.add(input1)

        update_dict(occupation_pair_count_exploitation, (oc1, oc2))
        update_dict(occupation1_count_exploitation, oc1)
        update_dict(occupation2_count_exploitation, oc2)
        update_dict(verb_count_exploitation, verb)
        update_dict(action_count_exploitation, action)



        
#             if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#                 if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if input1 not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add(input1)

    #                 print(pred1, pred2, pred3)
    #                 print(input1)
    #                 print(input2)
    #                 print(input3)

                    update_dict(occupation_pair_error_exploitation, (oc1, oc2))
                    update_dict(occupation1_error_exploitation, oc1)
                    update_dict(occupation2_error_exploitation, oc2)
                    update_dict(verb_error_exploitation, verb)
                    update_dict(action_error_exploitation, action)



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))

In [37]:
generate_test_sentences()

Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 16
Unique inputs: 28
Iterations: 30
------------------------------
Unique errors: 25
Unique inputs: 56
Iterations: 60
------------------------------
Unique errors: 42
Unique inputs: 86
Iterations: 90
------------------------------
Unique errors: 58
Unique inputs: 115
Iterations: 120
------------------------------
Unique errors: 72
Unique inputs: 143
Iterations: 150
------------------------------
Unique errors: 83
Unique inputs: 172
Iterations: 180
------------------------------
Unique errors: 96
Unique inputs: 198
Iterations: 210
------------------------------
Unique errors: 110
Unique inputs: 227
Iterations: 240
------------------------------
Unique errors: 125
Unique inputs: 256
Iterations: 270
------------------------------
Unique errors: 131
Unique inputs: 281
Iterations: 300
------------------------------
Unique errors: 143
Unique inputs: 308
Iterations: 330
------------------------------

Unique errors: 1203
Unique inputs: 2533
Iterations: 2880
------------------------------
Unique errors: 1213
Unique inputs: 2554
Iterations: 2910
------------------------------
Unique errors: 1225
Unique inputs: 2580
Iterations: 2940
------------------------------
Unique errors: 1241
Unique inputs: 2607
Iterations: 2970
------------------------------
1415
0.4716666666666667
Final Unique errors: 1254
Final Unique inputs: 2630


In [38]:

with open('saved_pickles/Exploitation/unique_input1_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_set_exploitation, handle)
    
with open('saved_pickles/Exploitation/unique_input1_error_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_error_set_exploitation, handle)

In [39]:
with open('saved_pickles/Exploitation/occupation_pair_count.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_count_exploitation, handle)
    
with open('saved_pickles/Exploitation/occupation1_count.pickle', 'wb') as handle:
    pickle.dump(occupation1_count_exploitation, handle)
    
with open('saved_pickles/Exploitation/occupation2_count.pickle', 'wb') as handle:
    pickle.dump(occupation2_count_exploitation, handle)
    
with open('saved_pickles/Exploitation/verb_count.pickle', 'wb') as handle:
    pickle.dump(verb_count_exploitation, handle)

with open('saved_pickles/Exploitation/action_count.pickle', 'wb') as handle:
    pickle.dump(action_count_exploitation, handle)

In [40]:
with open('saved_pickles/Exploitation/occupation_pair_error.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_error_exploitation, handle)
    
with open('saved_pickles/Exploitation/occupation1_error.pickle', 'wb') as handle:
    pickle.dump(occupation1_error_exploitation, handle)
    
with open('saved_pickles/Exploitation/occupation2_error.pickle', 'wb') as handle:
    pickle.dump(occupation2_error_exploitation, handle)
    
with open('saved_pickles/Exploitation/verb_error.pickle', 'wb') as handle:
    pickle.dump(verb_error_exploitation, handle)

with open('saved_pickles/Exploitation/action_error.pickle', 'wb') as handle:
    pickle.dump(action_error_exploitation, handle)

In [26]:
# occupations_2 = ['cashier',
# 'teacher',
# 'nurse',
# 'assistant',
# 'secretary',
# 'auditor',
# 'cleaner',
# 'receptionist',
# 'clerk',
# 'counselors',
# 'designer',
# 'hairdresser',
# 'attendant',
# 'writer',
# 'housekeeper',
# 'baker',
# 'accountant',
# 'editor',
# 'librarian',
# 'sewer']
# occupations_1.extend(["technician", "accountant", "engineer"])
# occupations_2.extend(["teacher", "librarian", "nurse", "paralegal"])

In [27]:
# verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
# verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [28]:
# error_rate_dict = get_error_rate_dict(occupation1_error, occupation1_count)
# for key in error_rate_dict:
#     print(key, error_rate_dict[key])

In [29]:
# error_rate_dict = get_error_rate_dict(occupation2_error, occupation2_count)
# for key in error_rate_dict:
#     print(key, error_rate_dict[key])

In [30]:
# error_rate_dict = get_error_rate_dict(occupation_pair_error, occupation_pair_count)
# for key in error_rate_dict:
#     print(key, error_rate_dict[key], occupation_pair_count[key])

In [31]:
# print(len(occupation_pair_count.keys()))
# print(len(occupation1_count.keys()) * len(occupation2_count.keys()) )